1. Unsupervised Learning Trading Strategy

In [9]:
pip install pandas numpy matplotlib statsmodels pandas_datareader datetime yfinance scikit-learn PyPortfolioOpt

Note: you may need to restart the kernel to use updated packages.


In [3]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = '2024-11-04'

start_date = pd.to_datetime(end_date)-pd.DateOffset(365*3) #Checks the last 3 years since specifide end_date.

df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['Date', 'Ticker']

df

[*********************100%%**********************]  503 of 503 completed


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2021-11-05 A       153.529037  156.410004  156.809998  152.429993  156.570007   
           AAPL    149.005600  151.279999  152.199997  150.059998  151.889999   
           ABBV    104.583336  117.180000  117.320000  115.500000  116.730003   
           ABNB    201.619995  201.619995  203.294998  185.944000  186.149994   
           ABT     117.572212  124.370003  127.320000  123.930000  127.080002   
...                       ...         ...         ...         ...         ...   
2024-11-01 XYL     119.459999  119.459999  122.440002  119.360001  121.440002   
           YUM     132.339996  132.339996  133.339996  131.820007  132.199997   
           ZBH     107.269997  107.269997  109.500000  107.139999  107.400002   
           ZBRA    384.640015  384.640015  386.119995  381.040009  381.970001   
           ZTS     181.949997  181.949997  183.110001  179.199997  179.979996   

Price                  Volume  
Date       Ticker              
2021-11-05 A        1775500.0  
           AAPL    65463900.0  
           ABBV     4635800.0  
           ABNB    27330500.0  
           ABT      7989800.0  
...                       ...  
2024-11-01 XYL      1925000.0  
           YUM      1888600.0  
           ZBH      1905700.0  
           ZBRA      523300.0  
           ZTS      2838600.0  

[374481 rows x 6 columns]

2. Calculate features and technical indicators for each stock:

Garman-Klass Volatility
RSI
Bollinger Bands
ATR
MACD
Dollar Volume

***REMEMBER TO CAPITALIZE COLUMN VALUES***

In [4]:
# Garman-Klass Volatility
df['garman_klass_vol'] = ((np.log(df['High']) - np.log(df['Low'])) ** 2) / 2 - (2 * np.log(2) - 1) * ((np.log(df['Adj Close']) - np.log(df['Open'])) ** 2)

# RSI Calculation with Mean-Fill for NaNs
df['rsi'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.rsi(close=x, length=14).fillna(x.mean()))

# Bollinger Bands (Low, Mid, High) with Mean-Fill for NaNs
df['bb_low'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 0].fillna(x.mean()))
df['bb_mid'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 1].fillna(x.mean()))
df['bb_high'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 2].fillna(x.mean()))

df

Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2021-11-05 A       153.529037  156.410004  156.809998  152.429993  156.570007   
           AAPL    149.005600  151.279999  152.199997  150.059998  151.889999   
           ABBV    104.583336  117.180000  117.320000  115.500000  116.730003   
           ABNB    201.619995  201.619995  203.294998  185.944000  186.149994   
           ABT     117.572212  124.370003  127.320000  123.930000  127.080002   
...                       ...         ...         ...         ...         ...   
2024-11-01 XYL     119.459999  119.459999  122.440002  119.360001  121.440002   
           YUM     132.339996  132.339996  133.339996  131.820007  132.199997   
           ZBH     107.269997  107.269997  109.500000  107.139999  107.400002   
           ZBRA    384.640015  384.640015  386.119995  381.040009  381.970001   
           ZTS     181.949997  181.949997  183.110001  179.199997  179.979996   

Price                  Volume  garman_klass_vol         rsi      bb_low  \
Date       Ticker                                                         
2021-11-05 A        1775500.0          0.000253  132.745778  132.745778   
           AAPL    65463900.0         -0.000042  173.138578  173.138578   
           ABBV     4635800.0         -0.004542  146.824268  146.824268   
           ABNB    27330500.0          0.001518  132.875306  132.875306   
           ABT      7989800.0         -0.001972  106.678209  106.678209   
...                       ...               ...         ...         ...   
2024-11-01 XYL      1925000.0          0.000220   20.721511    4.820718   
           YUM      1888600.0          0.000065   43.826079    4.888754   
           ZBH      1905700.0          0.000237   54.785410    4.624335   
           ZBRA      523300.0          0.000069   64.037550    5.886588   
           ZTS      2838600.0          0.000187   41.698570    5.185172   

Price                  bb_mid     bb_high  
Date       Ticker                          
2021-11-05 A       132.745778  132.745778  
           AAPL    173.138578  173.138578  
           ABBV    146.824268  146.824268  
           ABNB    132.875306  132.875306  
           ABT     106.678209  106.678209  
...                       ...         ...  
2024-11-01 XYL       4.891695    4.962671  
           YUM       4.903457    4.918160  
           ZBH       4.659489    4.694642  
           ZBRA      5.924520    5.962452  
           ZTS       5.235566    5.285961  

[374481 rows x 11 columns]

In [6]:
df['sma_20'] = df.groupby(level=1)['Adj Close'].transform(lambda x: x.rolling(window=20).mean().fillna(x.mean()))
df['sma_50'] = df.groupby(level=1)['Adj Close'].transform(lambda x: x.rolling(window=50).mean().fillna(x.mean()))
df['ema_20'] = df.groupby(level=1)['Adj Close'].transform(lambda x: x.ewm(span=20, adjust=False).mean().fillna(x.mean()))
df['ema_50'] = df.groupby(level=1)['Adj Close'].transform(lambda x: x.ewm(span=50, adjust=False).mean().fillna(x.mean()))

df

Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2021-11-05 A       153.529037  156.410004  156.809998  152.429993  156.570007   
           AAPL    149.005600  151.279999  152.199997  150.059998  151.889999   
           ABBV    104.583336  117.180000  117.320000  115.500000  116.730003   
           ABNB    201.619995  201.619995  203.294998  185.944000  186.149994   
           ABT     117.572212  124.370003  127.320000  123.930000  127.080002   
...                       ...         ...         ...         ...         ...   
2024-11-01 XYL     119.459999  119.459999  122.440002  119.360001  121.440002   
           YUM     132.339996  132.339996  133.339996  131.820007  132.199997   
           ZBH     107.269997  107.269997  109.500000  107.139999  107.400002   
           ZBRA    384.640015  384.640015  386.119995  381.040009  381.970001   
           ZTS     181.949997  181.949997  183.110001  179.199997  179.979996   

Price                  Volume  garman_klass_vol         rsi      bb_low  \
Date       Ticker                                                         
2021-11-05 A        1775500.0          0.000253  132.745778  132.745778   
           AAPL    65463900.0         -0.000042  173.138578  173.138578   
           ABBV     4635800.0         -0.004542  146.824268  146.824268   
           ABNB    27330500.0          0.001518  132.875306  132.875306   
           ABT      7989800.0         -0.001972  106.678209  106.678209   
...                       ...               ...         ...         ...   
2024-11-01 XYL      1925000.0          0.000220   20.721511    4.820718   
           YUM      1888600.0          0.000065   43.826079    4.888754   
           ZBH      1905700.0          0.000237   54.785410    4.624335   
           ZBRA      523300.0          0.000069   64.037550    5.886588   
           ZTS      2838600.0          0.000187   41.698570    5.185172   

Price                  bb_mid     bb_high      sma_20      sma_50      ema_20  \
Date       Ticker                                                               
2021-11-05 A       132.745778  132.745778  132.745778  132.745778  153.529037   
           AAPL    173.138578  173.138578  173.138578  173.138578  149.005600   
           ABBV    146.824268  146.824268  146.824268  146.824268  104.583336   
           ABNB    132.875306  132.875306  132.875306  132.875306  201.619995   
           ABT     106.678209  106.678209  106.678209  106.678209  117.572212   
...                       ...         ...         ...         ...         ...   
2024-11-01 XYL       4.891695    4.962671  132.261499  132.738140  130.440643   
           YUM       4.903457    4.918160  133.758501  134.282544  133.723181   
           ZBH       4.659489    4.694642  104.598499  106.949599  105.472877   
           ZBRA      5.924520    5.962452  373.165997  359.404999  372.518871   
           ZTS       5.235566    5.285961  186.894865  188.479268  185.711968   

Price                  ema_50        atr  
Date       Ticker                         
2021-11-05 A       153.529037   5.649414  
           AAPL    149.005600   4.557398  
           ABBV    104.583336  13.510775  
           ABNB    201.619995  11.064244  
           ABT     117.572212   8.256947  
...                       ...        ...  
2024-11-01 XYL     132.298532   2.664592  
           YUM     134.046096   2.057551  
           ZBH     106.537655   2.234337  
           ZBRA    361.013341   8.161989  
           ZTS     186.557855   3.739162  

[374481 rows x 16 columns]

In [7]:
df['atr'] = df.groupby(level=1).apply(lambda x: pandas_ta.atr(high=x['High'], low=x['Low'], close=x['Adj Close'], length=14)).reset_index(level=0, drop=True).fillna(method='bfill')

df

Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2021-11-05 A       153.529037  156.410004  156.809998  152.429993  156.570007   
           AAPL    149.005600  151.279999  152.199997  150.059998  151.889999   
           ABBV    104.583336  117.180000  117.320000  115.500000  116.730003   
           ABNB    201.619995  201.619995  203.294998  185.944000  186.149994   
           ABT     117.572212  124.370003  127.320000  123.930000  127.080002   
...                       ...         ...         ...         ...         ...   
2024-11-01 XYL     119.459999  119.459999  122.440002  119.360001  121.440002   
           YUM     132.339996  132.339996  133.339996  131.820007  132.199997   
           ZBH     107.269997  107.269997  109.500000  107.139999  107.400002   
           ZBRA    384.640015  384.640015  386.119995  381.040009  381.970001   
           ZTS     181.949997  181.949997  183.110001  179.199997  179.979996   

Price                  Volume  garman_klass_vol         rsi      bb_low  \
Date       Ticker                                                         
2021-11-05 A        1775500.0          0.000253  132.745778  132.745778   
           AAPL    65463900.0         -0.000042  173.138578  173.138578   
           ABBV     4635800.0         -0.004542  146.824268  146.824268   
           ABNB    27330500.0          0.001518  132.875306  132.875306   
           ABT      7989800.0         -0.001972  106.678209  106.678209   
...                       ...               ...         ...         ...   
2024-11-01 XYL      1925000.0          0.000220   20.721511    4.820718   
           YUM      1888600.0          0.000065   43.826079    4.888754   
           ZBH      1905700.0          0.000237   54.785410    4.624335   
           ZBRA      523300.0          0.000069   64.037550    5.886588   
           ZTS      2838600.0          0.000187   41.698570    5.185172   

Price                  bb_mid     bb_high      sma_20      sma_50      ema_20  \
Date       Ticker                                                               
2021-11-05 A       132.745778  132.745778  132.745778  132.745778  153.529037   
           AAPL    173.138578  173.138578  173.138578  173.138578  149.005600   
           ABBV    146.824268  146.824268  146.824268  146.824268  104.583336   
           ABNB    132.875306  132.875306  132.875306  132.875306  201.619995   
           ABT     106.678209  106.678209  106.678209  106.678209  117.572212   
...                       ...         ...         ...         ...         ...   
2024-11-01 XYL       4.891695    4.962671  132.261499  132.738140  130.440643   
           YUM       4.903457    4.918160  133.758501  134.282544  133.723181   
           ZBH       4.659489    4.694642  104.598499  106.949599  105.472877   
           ZBRA      5.924520    5.962452  373.165997  359.404999  372.518871   
           ZTS       5.235566    5.285961  186.894865  188.479268  185.711968   

Price                  ema_50        atr  
Date       Ticker                         
2021-11-05 A       153.529037   5.649414  
           AAPL    149.005600   4.557398  
           ABBV    104.583336  13.510775  
           ABNB    201.619995  11.064244  
           ABT     117.572212   8.256947  
...                       ...        ...  
2024-11-01 XYL     132.298532   2.664592  
           YUM     134.046096   2.057551  
           ZBH     106.537655   2.234337  
           ZBRA    361.013341   8.161989  
           ZTS     186.557855   3.739162  

[374481 rows x 16 columns]

In [13]:
import pandas_ta as ta

# Calculate MACD and Signal line
macd = ta.macd(df['Adj Close'], fast=12, slow=26, signal=9)

# Rename columns for ease of use
macd.rename(columns={
    'MACD_12_26_9': 'macd',
    'MACDh_12_26_9': 'macd_hist',
    'MACDs_12_26_9': 'macd_signal'
}, inplace=True)

# Fill missing values with the mean of the columns
df['macd'] = macd['macd'].fillna(macd['macd'].mean())
df['macd_signal'] = macd['macd_signal'].fillna(macd['macd_signal'].mean())

df

Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2021-11-05 A       153.529037  156.410004  156.809998  152.429993  156.570007   
           AAPL    149.005600  151.279999  152.199997  150.059998  151.889999   
           ABBV    104.583336  117.180000  117.320000  115.500000  116.730003   
           ABNB    201.619995  201.619995  203.294998  185.944000  186.149994   
           ABT     117.572212  124.370003  127.320000  123.930000  127.080002   
...                       ...         ...         ...         ...         ...   
2024-11-01 XYL     119.459999  119.459999  122.440002  119.360001  121.440002   
           YUM     132.339996  132.339996  133.339996  131.820007  132.199997   
           ZBH     107.269997  107.269997  109.500000  107.139999  107.400002   
           ZBRA    384.640015  384.640015  386.119995  381.040009  381.970001   
           ZTS     181.949997  181.949997  183.110001  179.199997  179.979996   

Price                  Volume  garman_klass_vol         rsi      bb_low  \
Date       Ticker                                                         
2021-11-05 A        1775500.0          0.000253  132.745778  132.745778   
           AAPL    65463900.0         -0.000042  173.138578  173.138578   
           ABBV     4635800.0         -0.004542  146.824268  146.824268   
           ABNB    27330500.0          0.001518  132.875306  132.875306   
           ABT      7989800.0         -0.001972  106.678209  106.678209   
...                       ...               ...         ...         ...   
2024-11-01 XYL      1925000.0          0.000220   20.721511    4.820718   
           YUM      1888600.0          0.000065   43.826079    4.888754   
           ZBH      1905700.0          0.000237   54.785410    4.624335   
           ZBRA      523300.0          0.000069   64.037550    5.886588   
           ZTS      2838600.0          0.000187   41.698570    5.185172   

Price                  bb_mid     bb_high      sma_20      sma_50      ema_20  \
Date       Ticker                                                               
2021-11-05 A       132.745778  132.745778  132.745778  132.745778  153.529037   
           AAPL    173.138578  173.138578  173.138578  173.138578  149.005600   
           ABBV    146.824268  146.824268  146.824268  146.824268  104.583336   
           ABNB    132.875306  132.875306  132.875306  132.875306  201.619995   
           ABT     106.678209  106.678209  106.678209  106.678209  117.572212   
...                       ...         ...         ...         ...         ...   
2024-11-01 XYL       4.891695    4.962671  132.261499  132.738140  130.440643   
           YUM       4.903457    4.918160  133.758501  134.282544  133.723181   
           ZBH       4.659489    4.694642  104.598499  106.949599  105.472877   
           ZBRA      5.924520    5.962452  373.165997  359.404999  372.518871   
           ZTS       5.235566    5.285961  186.894865  188.479268  185.711968   

Price                  ema_50        atr       macd  macd_signal  
Date       Ticker                                                 
2021-11-05 A       153.529037   5.649414  -0.000717    -0.000596  
           AAPL    149.005600   4.557398  -0.000717    -0.000596  
           ABBV    104.583336  13.510775  -0.000717    -0.000596  
           ABNB    201.619995  11.064244  -0.000717    -0.000596  
           ABT     117.572212   8.256947  -0.000717    -0.000596  
...                       ...        ...        ...          ...  
2024-11-01 XYL     132.298532   2.664592 -17.090030   -18.498009  
           YUM     134.046096   2.057551 -15.051852   -17.808778  
           ZBH     106.537655   2.234337 -15.283343   -17.303691  
           ZBRA    361.013341   8.161989   6.835841   -12.475785  
           ZTS     186.557855   3.739162   7.918781    -8.396872  

[374481 rows x 18 columns]

In [20]:
# Rate of Change
df['roc'] = df.groupby(level=1)['Adj Close'].transform(lambda x: ((x - x.shift(12)) / x.shift(12)).fillna(x.mean()))

# Volume Change
df['vol_change'] = df['Volume'].pct_change().fillna(df['Volume'].mean())

# Daily Returns
df['daily_return'] = df['Adj Close'].pct_change().fillna(df['Adj Close'].mean())

# Lagged Returns
df['lagged_return_1'] = df['daily_return'].shift(1).fillna(df['daily_return'].mean())
df['lagged_return_5'] = df['daily_return'].shift(5).fillna(df['daily_return'].mean())
df['lagged_return_10'] = df['daily_return'].shift(10).fillna(df['daily_return'].mean())

# Historical Volatility
df['historical_volatility'] = df.groupby(level=1)['daily_return'].transform(lambda x: x.rolling(window=30).std().fillna(x.mean()))

df

Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2021-11-05 A       153.529037  156.410004  156.809998  152.429993  156.570007   
           AAPL    149.005600  151.279999  152.199997  150.059998  151.889999   
           ABBV    104.583336  117.180000  117.320000  115.500000  116.730003   
           ABNB    201.619995  201.619995  203.294998  185.944000  186.149994   
           ABT     117.572212  124.370003  127.320000  123.930000  127.080002   
...                       ...         ...         ...         ...         ...   
2024-11-01 XYL     119.459999  119.459999  122.440002  119.360001  121.440002   
           YUM     132.339996  132.339996  133.339996  131.820007  132.199997   
           ZBH     107.269997  107.269997  109.500000  107.139999  107.400002   
           ZBRA    384.640015  384.640015  386.119995  381.040009  381.970001   
           ZTS     181.949997  181.949997  183.110001  179.199997  179.979996   

Price                  Volume  garman_klass_vol         rsi      bb_low  \
Date       Ticker                                                         
2021-11-05 A        1775500.0          0.000253  132.745778  132.745778   
           AAPL    65463900.0         -0.000042  173.138578  173.138578   
           ABBV     4635800.0         -0.004542  146.824268  146.824268   
           ABNB    27330500.0          0.001518  132.875306  132.875306   
           ABT      7989800.0         -0.001972  106.678209  106.678209   
...                       ...               ...         ...         ...   
2024-11-01 XYL      1925000.0          0.000220   20.721511    4.820718   
           YUM      1888600.0          0.000065   43.826079    4.888754   
           ZBH      1905700.0          0.000237   54.785410    4.624335   
           ZBRA      523300.0          0.000069   64.037550    5.886588   
           ZTS      2838600.0          0.000187   41.698570    5.185172   

Price                  bb_mid  ...        atr       macd  macd_signal  \
Date       Ticker              ...                                      
2021-11-05 A       132.745778  ...   5.649414  -0.000717    -0.000596   
           AAPL    173.138578  ...   4.557398  -0.000717    -0.000596   
           ABBV    146.824268  ...  13.510775  -0.000717    -0.000596   
           ABNB    132.875306  ...  11.064244  -0.000717    -0.000596   
           ABT     106.678209  ...   8.256947  -0.000717    -0.000596   
...                       ...  ...        ...        ...          ...   
2024-11-01 XYL       4.891695  ...   2.664592 -17.090030   -18.498009   
           YUM       4.903457  ...   2.057551 -15.051852   -17.808778   
           ZBH       4.659489  ...   2.234337 -15.283343   -17.303691   
           ZBRA      5.924520  ...   8.161989   6.835841   -12.475785   
           ZTS       5.235566  ...   3.739162   7.918781    -8.396872   

Price                     roc    vol_change  daily_return  lagged_return_1  \
Date       Ticker                                                            
2021-11-05 A       132.745778  6.071164e+06    174.332876         1.540848   
           AAPL    173.138578  3.587068e+01     -0.029463       174.332876   
           ABBV    146.824268 -9.291854e-01     -0.298125        -0.029463   
           ABNB    132.875306  4.895530e+00      0.927841        -0.298125   
           ABT     106.678209 -7.076599e-01     -0.416862         0.927841   
...                       ...           ...           ...              ...   
2024-11-01 XYL      -0.130947 -9.080676e-01      0.039234         0.723647   
           YUM      -0.007574 -1.890909e-02      0.107818         0.039234   
           ZBH       0.017163  9.054326e-03     -0.189436         0.107818   
           ZBRA      0.024341 -7.254027e-01      2.585719        -0.189436   
           ZTS      -0.062018  4.424422e+00     -0.526960         2.585719   

Price